In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import tqdm
import torchnet.meter as meter


In [5]:
LAMBDA = 2048
BATCH_SIZE = 4
# DATA_DIR = pathlib.Path('/data2/jatin/vimeo_septuplet/sequences')
DEVICE = torch.device('cpu')
DEVICE

device(type='cpu')

In [36]:
video_net = DCVC_net()

In [37]:
optimizer = torch.optim.Adam(video_net.parameters(), lr=1e-4)

In [8]:
criterion = torch.nn.MSELoss()

In [38]:
def count_parameters(model):
    """Return number of parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(video_net)} trainable parameters')

The model has 13439734 trainable parameters


In [11]:
img1 = plt.imread("../uvg/Beauty_PNG_1024/im00001.png")
img2 = plt.imread("../uvg/Beauty_PNG_1024/im00002.png")
img3 = plt.imread("../uvg/Beauty_PNG_1024/im00003.png")

print(img1.shape)

(1024, 1920, 3)


In [12]:
img1 = img1[:128, :128, :]
img2 = img2[:128, :128, :]
img3 = img3[:128, :128, :]

In [13]:
img1 = torch.FloatTensor(img1.transpose(2, 0, 1)).unsqueeze(0)
img2 = torch.FloatTensor(img2.transpose(2, 0, 1)).unsqueeze(0)
img3 = torch.FloatTensor(img3.transpose(2, 0, 1)).unsqueeze(0)

In [39]:
preds_dict = video_net(img1, img2, "train_compress")

In [40]:
loss = criterion(preds_dict["recon_image"], img2)
loss.backward()
optimizer.step()